In [ ]:
import gcsfs
import dask_gateway
import dask.dataframe as dd
import pandas as pd
import json

In [ ]:
token = json.load(open("credentials.json"))
fs = gcsfs.GCSFileSystem(token=token)
storage_options={"token": token}

In [ ]:
gateway = dask_gateway.Gateway()
if len(running_clusters := gateway.list_clusters())>0:
    cluster = gateway.connect(running_clusters[0].name)
else:
    cluster = gateway.new_cluster(conda_environment="pycon2023/pycon2023-tutorial", profile="Medium Worker")
    cluster.adapt(1,100)

In [ ]:
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
with open('dtypes.json', 'r') as f:
    dtypes = json.load(f)

In [ ]:
ddf = dd.read_csv("gcs://quansight-datasets/airline-ontime-performance/csv/*", 
                  dtype=dtypes, 
                  parse_dates=['FL_DATE'], 
                  storage_options=storage_options,
                 )

In [ ]:
#ddf = ddf.sort_values(["FL_DATE", "OP_CARRIER"]).persist()

In [ ]:
(
    ddf.sort_values(["FL_DATE", "OP_CARRIER"])
       .to_parquet("gcs://quansight-datasets/airline-ontime-performance/sorted/full_dataset.parquet", 
            storage_options=storage_options,
            write_index=False,
            )
)

In [ ]:
ddf.to_parquet("gcs://quansight-datasets/airline-ontime-performance/sorted/full_dataset.parquet", 
            storage_options=storage_options,
            write_index=True,
            )

In [ ]:
ddf = dd.read_parquet("gcs://quansight-datasets/airline-ontime-performance/sorted/full_dataset.parquet")

In [ ]:
ddf.head()

In [ ]:
ddf.to_parquet("gcs://quansight-datasets/airline-ontime-performance/sorted/parquet_by_year", 
            partition_on=['YEAR'], 
            storage_options=storage_options,
            write_index=False,
            )

In [ ]:
cluster.shutdown()